# Проект Дашборд конверсий

In [1]:
import pandas as pd
import requests

## Получаем данные о регистрациях и посещениях по API

In [6]:
req_visits = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
df_visits = pd.DataFrame(req_visits.json())

req_registrations = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
df_registrations = pd.DataFrame(req_registrations.json())

                        datetime                       email platform  \
0  Wed, 01 Mar 2023 07:40:13 GMT           ebyrd@example.org      web   
1  Wed, 01 Mar 2023 13:14:00 GMT    knightgerald@example.org      web   
2  Wed, 01 Mar 2023 03:05:50 GMT  cherylthompson@example.com      web   
3  Wed, 01 Mar 2023 00:04:47 GMT       halldavid@example.org      web   
4  Wed, 01 Mar 2023 18:31:52 GMT        denise86@example.net      web   

  registration_type                               user_id  
0            google  2e0f6bb8-b029-4f45-a786-2b53990d37f1  
1             email  f007f97c-9d8b-48b5-af08-119bb8f6d9b6  
2             apple  24ff46ae-32b3-4a74-8f27-7cf0b8f32f15  
3             email  3e9914e1-5d73-4c23-b25d-b59a3aeb2b60  
4            google  27f875fc-f8ce-4aeb-8722-0ecb283d0760  


In [9]:
#Предварительный анализ
df_visits.describe

<bound method NDFrame.describe of                              datetime platform  \
0       Wed, 01 Mar 2023 10:36:22 GMT      web   
1       Wed, 01 Mar 2023 06:25:00 GMT      web   
2       Wed, 01 Mar 2023 10:26:13 GMT      web   
3       Wed, 01 Mar 2023 12:33:06 GMT      web   
4       Wed, 01 Mar 2023 01:38:35 GMT      web   
...                               ...      ...   
263454  Tue, 29 Aug 2023 10:43:29 GMT      web   
263455  Tue, 29 Aug 2023 19:29:16 GMT      web   
263456  Tue, 29 Aug 2023 12:51:11 GMT      web   
263457  Tue, 29 Aug 2023 21:45:08 GMT      web   
263458  Wed, 30 Aug 2023 19:19:04 GMT      web   

                                               user_agent  \
0       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
1       Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...   
2       Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...   
3       Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...   
4       Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   

In [8]:
#Предварительный анализ
df_registrations.describe

<bound method NDFrame.describe of                             datetime                       email platform  \
0      Wed, 01 Mar 2023 07:40:13 GMT           ebyrd@example.org      web   
1      Wed, 01 Mar 2023 13:14:00 GMT    knightgerald@example.org      web   
2      Wed, 01 Mar 2023 03:05:50 GMT  cherylthompson@example.com      web   
3      Wed, 01 Mar 2023 00:04:47 GMT       halldavid@example.org      web   
4      Wed, 01 Mar 2023 18:31:52 GMT        denise86@example.net      web   
...                              ...                         ...      ...   
21831  Thu, 31 Aug 2023 07:32:08 GMT         ikelley@example.net      ios   
21832  Thu, 31 Aug 2023 23:25:50 GMT     brittanycox@example.net      ios   
21833  Thu, 31 Aug 2023 19:29:18 GMT           aryan@example.net      ios   
21834  Thu, 31 Aug 2023 19:38:54 GMT       emartinez@example.net      ios   
21835  Thu, 31 Aug 2023 08:36:05 GMT      terriperry@example.net      ios   

      registration_type                  